# Kalman Filter - Modelling of COVID-19  (UNDER DEVELOPMENT)

**Authors:** J.H. Cao 
<br>
**Supervisors:** prof.dr.ir. N.A.W. van Riel, dr. D. Bosnacki
<br>
**Deparment:** Biomedical Engineering, Computational Biology Group at Eindhoven University of Technology

This notebook contain the codes to model and simulate an epidemic. The **Kalman filter** is a mathematical model that can be used as a predictor or filter. This means that this Kalman Filter can also be applied on the SEIR models or the Logistic Growth Curve, as a filter. But in this section, Kalman Filter is used as a predictor, to forecast the spread of COVID-19 in the Netherlands. Since, the Kalman Filter is ideal for systems that are continuously changing, the spread of COVID-19, which is time-dependent can also be seen as a continuously changing system.
<br>
<br>
This Kalman Filter for estimation will be applied on the:
1. A self-defined **exponential growth system with randomization on the growth parameter** (based on the available data - *daily confirmed cases*)
2. The **SIR/SEIR** model, to take uncertainties into account and use this as a filter on the dynamic systems of SIR/SEIR models.

The Kalman Filter's application mentioned in **(2)** might not be used for estimation or as a predictor, but more or less as a filter for the dynamic system by taking uncertainties of that system into account. Another possibility is to use the Kalman Filter to estimate and update the parameters found in these differential equations. 

## Imports

### Packages

This section is for the import of the important python packages needed to develop the Kalman Filter.

In [1]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit # This package is necessary for the logistic growth curve fit 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### Data - Number of recorded cases in the Netherlands

This section is for the import of the data regarding the number of recorded cases of COVID-19 in the Netherlands.

In [2]:
# The newest file containing the number of (+) cases in the Netherlands

covid19_NL_file = './data/rivm_corona_in_nl_daily.txt'

df_covid =  pd.read_csv(covid19_NL_file, parse_dates=['Datum']).set_index("Datum")

In [3]:
# Adding a column of timestep for the curve_fit to the dataframe

df_covid['Timestep'] = [timestep for timestep in range(0, len(df_covid))]


# Re-arrange the columns 

cols = df_covid.columns.to_list()
cols = cols[1:] + cols[:-1]

df_covid = df_covid[cols]

In [4]:
# Changing the column name from dutch to english

df_covid = df_covid.rename(columns={"Aantal":"Total Cases"})

In [138]:
# Adding an extra column called "New Cases", which shows the new confirmed cases on that specific day
df_covid['New Cases'] = df_covid[['Total Cases']].diff()
df_covid['New Cases'].iloc[0] = 0

#### Final DataFrame

In [6]:
df_covid.head()

,Timestep,Total Cases,New Cases
Datum,,,
2020-02-27,0,1,0.0
2020-02-28,1,2,1.0
2020-02-29,2,7,5.0
2020-03-01,3,10,3.0
2020-03-02,4,18,8.0


#### Loading the logistic predictions

In [125]:
df_logpred = pd.read_csv('./data/log_growth_predictions-longterm.csv',parse_dates=['Datum']).set_index("Datum")

#### Data for the training of the model / KF

In [7]:
t = np.array(df_covid['Timestep'])  # timestep for predictions
y_true = np.array(df_covid['Total Cases'])  # the actual confirmed cases 

<br>

## (1) Kalman Filter for estimation -  Self-defined exponential growth function

The growth factor that is used for this function, is calculated by **dividing** the `new confirmed cases on day x` by the <br> `new confirmed cases on day x-1`. Afterwards, random uncertaintity is applied on this growth factor. 
<br>
<br>
Thus, once again, the **new_total_cases**`(day x)` = **total_cases**`(day x-1)` x `estimated growth factor (with randomization)`

### Kalman Filter prediction and update function

Since the process and measurement noise are **unknown**, it is assumed that these noise are **equal to zero**. 

In the function for the Kalman Filter, it can be seen that there are quite some variables. The definition for these variables are:

1. **y_pred** = the predicted measurement (total confirmed cases) at the given time (time in days)
2. **x_pred** = the best estimate of the state (the total confirmed cases) at given time (time in days)
<br><br>
3. **P**      = the state covariance matrix
4. **A** = state transition matrix 
5. **H** = observation matrix (relating **x_pred** and **y_true**)
6. **Q** = process covariance noise 
7. **R** = measurement noise covariance
8. **S** = innovation covariance matrix 
9. **K** = Kalman Gain 

#### Kalman Filter:

In [91]:
def kf_filter(x_pred, y_pred, P, A, H, Q, R):
    
    
    for k in range(0, len(x_pred)-1):
        
        # Prediction step
        x_pred[k+1] = A*x_pred[k]
        P = A*P*A.T + Q
        
        # Calculating the Kalman Gain 
        S = H*P*H.T + R
        K = P * H.T * np.linalg.inv(S)
        
        # Updating the "a priori state" with measurements to obtain the "a posteriori state"
        x_pred[k+1] = x_pred[k+1] + K*(y_pred[k+1] - H*x_pred[k+1])
        P = (1-K*H)*P
    
    return np.round(x_pred)  # rounding off the whole numbers 


### Prediction till 21/05/2020 (dd/mm/yyy)

In [92]:
# Making extra array space 10-day ahead prediction 
t_extra = np.arange(10)
x_est = np.zeros(len(t_extra))


# Taking the latest data available into account 

x_est[0] = y_true[-1]
x_est[1] = x_est[0] + ((y_true[-1] - y_true[-2]) * (np.random.randint(115, 122)/100)) # random growth parameter between 1.15 and 1.22

In [93]:
# Making the predictions for the upcoming 10 days and random growth parameter each time 

for i in range(2, len(x_est)):
    x_est[i] = x_est[i-1] + ((x_est[i-1] - x_est[i-2]) * (np.random.randint(115, 122)/100)) 
    

# Assign it to new array y_pred

y_pred = np.append(y_true, x_est[1:])

#### Kalman Filter Estimation 

In [94]:
# Making empty array for Kalman 
x_pred = np.zeros(len(y_pred))
t_new = np.arange(len(x_pred))

In [95]:
# Initial values of for the Kalman Filter 

x_pred[0] = 1
P = np.array([[0.1]])
A = np.array([[1.006]]) # assuming a constant state transition 
H = np.array([[1]])     # no drastic effect of the observation matrix  
Q = np.array([[0.1]])   # relative large process noise 
R = np.array([[0.01]])  # some measurement noise 

In [96]:
x_pred = kf_filter(x_pred, y_pred, P, A, H, Q, R)

#### Assigning the predictions to DataFrame - Predictions till 21/05/2020

In [115]:
# creating index as date starting from 27-02-2020
dates_ = pd.to_datetime(t_new, unit='D',origin=pd.Timestamp('2020-02-27')) 

# create dataframe with date as index and display the last few rows 
kf_dataframe = pd.DataFrame({'Kalman Filter Prediction': x_pred}).set_index(dates_)
kf_dataframe.tail(9)

,Kalman Filter Prediction
2020-05-13,43218.0
2020-05-14,43483.0
2020-05-15,43798.0
2020-05-16,44173.0
2020-05-17,44604.0
2020-05-18,45121.0
2020-05-19,45746.0
2020-05-20,46484.0
2020-05-21,47335.0


In [116]:
# Kalman 2-days ahead estimations
kf_pred_2days = kf_dataframe.iloc[75:77].copy()
kf_pred_2days.iloc[0] = 42980.0

In [117]:
kf_pred_2days

,Kalman Filter Prediction
2020-05-12,42980.0
2020-05-13,43218.0


### Comparing the results of Kalman Filter with the Logistic Growth 

The `Kalman Filter` predictions will be compared with the `Logistic Growth predictions`, on both short and long term predictions. Furthermore, these resutls will also be compared to the `actual confirmed cases`.

#### Short-term comparison (the upcoming two days) - Kalman Filter VS Logistic Growth

In [126]:
kf_vs_log_short_term = kf_pred_2days.join(df_logpred.iloc[75:77])
kf_vs_log_short_term

,Kalman Filter Prediction,Logistic Growth prediction
2020-05-12,42980.0,42124
2020-05-13,43218.0,42220


#### Long-term comparison (the upcoming two days) - Kalman Filter VS Logistic Growth

In [129]:
kf_vs_log_long_term = kf_dataframe.iloc[75:].join(df_logpred.iloc[75:])
kf_vs_log_long_term

,Kalman Filter Prediction,Logistic Growth prediction
2020-05-12,42990.0,42124.0
2020-05-13,43218.0,42220.0
2020-05-14,43483.0,42307.0
2020-05-15,43798.0,42384.0
2020-05-16,44173.0,42453.0
2020-05-17,44604.0,42515.0
2020-05-18,45121.0,42570.0
2020-05-19,45746.0,42620.0
2020-05-20,46484.0,42664.0
2020-05-21,47335.0,NaN


#### Comparison with actual confirmed cases - Actual confirmed cases VS Kalman Filter VS Logistic Growth

In [134]:
# Generating the values of confirmed cases 
cases_array = np.array([df_covid["Total Cases"].iloc[-1], "?"])
dates_comparison = pd.to_datetime([0,1], unit='D',origin=pd.Timestamp('2020-05-12')) 

# Creating a DataFrame for comparison 
df_comparison = pd.DataFrame({'Confirmed Cases': cases_array}).set_index(dates_comparison)

In [136]:
df_comparison.join(kf_pred_2days.join(df_logpred[75:77]))

,Confirmed Cases,Kalman Filter Prediction,Logistic Growth prediction
2020-05-12,42984,42980.0,42124
2020-05-13,?,43218.0,42220
